In [2]:
from fastai.vision.all import *
from pathlib import Path
import os

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
path = Path("/kaggle/input/full-breast3-ihc-3072")

def get_y_fn(x):
    return Path(str(x).replace("images", "masks"))

In [4]:
codes = ['background', 'tissue']

data_block = DataBlock(
    blocks=(ImageBlock, MaskBlock(codes)),
    get_items=get_image_files,
    splitter=GrandparentSplitter(train_name='train', valid_name='test'),
    get_y=get_y_fn,
    batch_tfms=[*aug_transforms(), Normalize.from_stats(*imagenet_stats)]
)

dls = data_block.dataloaders(path, path='train/images', bs=16)

In [5]:
learn = unet_learner(dls, resnet34, metrics=JaccardCoeff)
learn.fine_tune(10)

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 191MB/s] 


epoch,train_loss,valid_loss,jaccard_coeff,time
0,0.395095,0.379577,0.722505,04:29


epoch,train_loss,valid_loss,jaccard_coeff,time
0,0.241989,0.198098,0.887279,04:42
1,0.189591,0.136804,0.926100,04:42
2,0.174151,0.122047,0.932700,04:41
3,0.178688,0.228155,0.903263,04:41
4,0.142033,3.495093,0.776885,04:41
5,0.139382,0.342632,0.911729,04:41
6,0.106883,0.289321,0.914906,04:42
7,0.111370,0.348978,0.865073,04:43
8,0.086896,0.250159,0.939776,04:44
9,0.069063,0.326152,0.915401,04:44


In [12]:
learn.save('3072-model')


Path('train/images/models/3072-model.pth')

In [14]:
learn.export('3072-model.pkl')

In [20]:
!zip -r train.zip ./train

updating: train/ (stored 0%)
  adding: train/images/ (stored 0%)
  adding: train/images/3072-model.pkl (deflated 14%)
  adding: train/images/models/ (stored 0%)
  adding: train/images/models/3072-model.pth (deflated 8%)


In [6]:
test_path = Path('/kaggle/input/test-img/test')

def get_test_items(path):
    files = get_image_files(path)
    return files, [f.name for f in files]

test_files, test_filenames = get_test_items(test_path)

test_dl = learn.dls.test_dl(test_files, with_labels=False)

In [7]:
preds, _ = learn.get_preds(dl=test_dl)

In [8]:
!mkdir test_predictions

In [9]:
threshold = 0.5

binary_masks = preds.argmax(dim=1)  
binary_masks = (binary_masks == 1) 
binary_masks = binary_masks.float() 

In [10]:
for i, filename in enumerate(test_filenames):
    pred = binary_masks[i]  # Get the prediction for this file
    # Process 'pred' as needed, e.g., convert to binary mask, save to file, etc.
    # Example: save the prediction as an image
    pred_img = Image.fromarray(pred.numpy().astype(np.uint8) * 255)
    pred_img.save(f'./test_predictions/{filename}')
    

In [11]:
!zip -r test_preds.zip /kaggle/working/test_predictions

  adding: kaggle/working/test_predictions/ (stored 0%)
  adding: kaggle/working/test_predictions/TCGA-A2-A0CV-01Z-00-DX1_49152_33792.png (deflated 82%)
  adding: kaggle/working/test_predictions/TCGA-A2-A0CV-01Z-00-DX1_129024_33792.png (deflated 82%)
  adding: kaggle/working/test_predictions/TCGA-A2-A0CV-01Z-00-DX1_147456_27648.png (deflated 82%)
  adding: kaggle/working/test_predictions/TCGA-A2-A0CV-01Z-00-DX1_95232_64512.png (deflated 66%)
  adding: kaggle/working/test_predictions/TCGA-A2-A0CV-01Z-00-DX1_46080_49152.png (deflated 82%)
  adding: kaggle/working/test_predictions/TCGA-A2-A0CV-01Z-00-DX1_89088_58368.png (deflated 48%)
  adding: kaggle/working/test_predictions/TCGA-A2-A0CV-01Z-00-DX1_98304_70656.png (deflated 65%)
  adding: kaggle/working/test_predictions/TCGA-A2-A0CV-01Z-00-DX1_113664_30720.png (deflated 82%)
  adding: kaggle/working/test_predictions/TCGA-A2-A0CV-01Z-00-DX1_27648_70656.png (deflated 44%)
  adding: kaggle/working/test_predictions/TCGA-A2-A0CV-01Z-00-DX1_150